# Compulsory Assignment 3 Teststrecke

Leonard Brenk, Finn Federsan, Felix Wltschek

## Question 1

In [2]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
# Load datasets
true_csv = pd.read_csv("True.csv")
fake_csv = pd.read_csv("Fake.csv")
fake_csv['label'] = 0
true_csv['label'] = 1
df = pd.concat([fake_csv, true_csv], ignore_index=True)

In [4]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


### 1.1. Preprocess

In [5]:
# Preprocessing
df['text'] = df['title'] + ' ' + df['text']
df = df.drop(['title', 'subject', 'date'], axis=1)
df['text'] = df['text'].apply(lambda x: re.sub(f"[{re.escape(string.punctuation)}]", " ", x))
df['text'] = df['text'].apply(lambda x: re.sub("\s+", " ", x))
df['text'] = df['text'].apply(lambda x: x.lower())

### 1.2 Stoppwords

In [6]:
# Removing stopwords
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words and len(word) > 2]))


### 1.3 Split

In [7]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)


### 1.4 Tokenize

In [8]:
# Tokenizing and padding
max_len = 250
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')
vocab_size = len(tokenizer.word_index) + 1

### 1.5 Model 

In [9]:
# Building model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



### 1.6 Training

In [10]:
# Training model
model.fit(X_train_padded, y_train, batch_size=64, epochs=2, validation_split=0.2)


2023-04-07 15:45:33.612104: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2
449/449 [==============================] - 69s 149ms/step - loss: 0.0752 - accuracy: 0.9700 - val_loss: 0.0204 - val_accuracy: 0.9940
Epoch 2/2
449/449 [==============================] - 66s 146ms/step - loss: 0.0068 - accuracy: 0.9982 - val_loss: 0.0156 - val_accuracy: 0.9953


### 1.7 Evaluate accuracy

In [11]:
# Evaluate the model on test data, print the accuracy, and return the loss and accuracy values

loss, accuracy = model.evaluate(X_test_padded, y_test)
print("accuracy: " + str(accuracy))
print("loss: " + str(loss))


281/281 [==============================] - 12s 42ms/step - loss: 0.0198 - accuracy: 0.9949
accuracy: 0.9948775172233582
loss: 0.019771970808506012
